In [1]:
import pandas as pd

data = pd.read_excel('train_set_spam.xlsx')
data = data.dropna(subset=['review'])

In [2]:
data['label'].value_counts()

0.0    547
1.0    233
Name: label, dtype: int64

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15806 entries, 0 to 15878
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  15806 non-null  object 
 1   label   780 non-null    float64
dtypes: float64(1), object(1)
memory usage: 370.5+ KB


In [4]:
from sklearn.model_selection import train_test_split

data2 = data.dropna(subset=['label'])
data2['label'] = data2['label'].astype(int)

C:\Users\tymot\AppData\Local\Temp\ipykernel_21304\761110511.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['label'] = data2['label'].astype(int)


In [5]:
# Wybieramy losowe próbki z klasy 0
class_0 = data2[data2['label'] == 1]

# Wybieramy losowe próbki z klasy 1
class_1 = data2[data2['label'] == 0].sample(len(class_0))

# Łączymy wybrane próbki z obu klas w jedną ramkę danych
undersampled_df = pd.concat([class_0, class_1])

data2 = undersampled_df 

In [6]:
X = data2[['review','label']]
X_train, X_test = train_test_split(X,train_size=0.8, random_state=0)

In [7]:
X_test.head(50)

,review,label
443,veri gud gaem,0
621,idk man. just doesn't grab me.,0
487,e frumos foc,1
332,Much Wow,1
712,BEST FPS EVER,0
89,wow!!!!!!!!!!!,1
279,"dont play this ame, it will ruin your life, if...",0
78,i hate this game with a passion but i cant get...,0
116,niceeeee,0
768,taeg,1


In [8]:
from datasets import Dataset
aa = Dataset.from_pandas(X_train)
aa = aa.remove_columns(["__index_level_0__"])
aa

Dataset({
    features: ['review', 'label'],
    num_rows: 372
})

In [9]:
a2 = Dataset.from_pandas(X_test)
a2 = a2.remove_columns(["__index_level_0__"])
a2

Dataset({
    features: ['review', 'label'],
    num_rows: 94
})

In [10]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import create_optimizer
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def preprocess_function(examples):
    return tokenizer(examples["review"], truncation=True)

toka = aa.map(preprocess_function, batched=True)
toka2 = a2.map(preprocess_function, batched=True)

Map:   0%|          | 0/372 [00:00<?, ? examples/s]

Map:   0%|          | 0/94 [00:00<?, ? examples/s]

In [11]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 5
batches_per_epoch = len(toka) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=1e-4, num_warmup_steps=0, num_train_steps=total_train_steps)

In [12]:
import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [14]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

tf_train_set = model.prepare_tf_dataset(
    toka,
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    toka2,
    shuffle=False,
    batch_size=4,
    collate_fn=data_collator,
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [15]:
import tensorflow as tf
model.compile(optimizer=optimizer)

from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)



No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [16]:
with tf.device('/GPU:0'):
    model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[metric_callback])

Epoch 1/3
93/93 [==============================] - 17s 102ms/step - loss: 0.4622 - val_loss: 0.3861 - accuracy: 0.8511
Epoch 2/3
93/93 [==============================] - 5s 53ms/step - loss: 0.2520 - val_loss: 0.3901 - accuracy: 0.8404
Epoch 3/3
93/93 [==============================] - 5s 54ms/step - loss: 0.2315 - val_loss: 0.3901 - accuracy: 0.8404


In [19]:

test_texts = ""
encoded_test_texts = tokenizer(test_texts, padding=True, truncation=True)

input_ids = np.array(encoded_test_texts['input_ids'])
attention_mask = np.array(encoded_test_texts['attention_mask'])


predictions = model.predict([input_ids, attention_mask])
predicted_probabilities = np.array(predictions.logits)
predicted_probabilities = np.exp(predicted_probabilities) / np.sum(np.exp(predicted_probabilities), axis=1, keepdims=True)

print(f"Text: {test_texts}\nNot spam: {predicted_probabilities[0][0]}, Spam: {predicted_probabilities[0][1]}\n")


1/1 [==============================] - 0s 40ms/step
Text: asdasdasddasd asdsdsdasd
Not spam: 0.07249745726585388, Spam: 0.9275025129318237



In [93]:
num_params = model.count_params()
print(num_params)

66955010
